# Data Cleaning

In this notebook we'll take the raw data downloaded earlier and clean it to remove the parts we don't need. This will drastically reduce the file size and allow us to condense everything down to a single file, rather than the massive file structure that it is downloaded as.


In [1]:
# basic imports
import pandas as pd
import os, json, pickle, math, lzma
from json import JSONDecodeError
from datetime import datetime, timezone
from multiprocessing.pool import ThreadPool

# Note: I run this notebook on a 16-core hyperthreaded CPU, so I have 32 available CPUs. Adjust this number as necessary to reflect the number of processors you have if you'd like to run this on a different machine
processors = 24

Let's take a look at one of our bill files to see what they look like, to give us a better idea of what we do and don't need to retrieve.

In [2]:
with open(r"H:\Programming\Big Input Data Stuff\LegiScan\input\CO\2024-2024_Regular_Session\bill\HB1130.json", 'r') as f:
    print(json.dumps(json.loads(f.read()), indent=2))

{
  "bill": {
    "bill_id": 1830799,
    "change_hash": "890f49f11280b1634d428686b5fa6564",
    "session_id": 2116,
    "session": {
      "session_id": 2116,
      "state_id": 6,
      "year_start": 2024,
      "year_end": 2024,
      "prefile": 0,
      "sine_die": 1,
      "prior": 0,
      "special": 0,
      "session_tag": "Regular Session",
      "session_title": "2024 Regular Session",
      "session_name": "2024 Regular Session"
    },
    "url": "https://legiscan.com/CO/bill/HB1130/2024",
    "state_link": "https://leg.colorado.gov/bills/hb24-1130",
    "completed": 1,
    "status": 4,
    "status_date": "2024-05-31",
    "progress": [
      {
        "date": "2024-01-29",
        "event": 1
      },
      {
        "date": "2024-01-29",
        "event": 9
      },
      {
        "date": "2024-02-14",
        "event": 10
      },
      {
        "date": "2024-02-14",
        "event": 9
      },
      {
        "date": "2024-02-23",
        "event": 2
      },
      {
       

Ok that's a lot of information, most of it useless. Let's take out what we care about.


In [3]:
statuses = [
    "N\\A",
    "Intro",
    "Engrossed",
    "Enrolled",
    "Passed",
    "Vetoed",
    "Failed",
    "Override",
    "Chaptered",
    "Refer",
    "Report Pass",
    "Report DNP",
    "Draft",
]

with open(r"H:\Programming\Big Input Data Stuff\LegiScan\input\CO\2024-2024_Regular_Session\bill\HB1130.json", 'r') as f:
    bill = json.loads(f.read())

bill_id = bill["bill"]["bill_id"]
status = statuses[bill["bill"]["status"]]
bill_name = bill["bill"]["title"]
bill_number = bill["bill"]["bill_number"]
bill_state = bill["bill"]["state"]
bill_desc = bill["bill"]["description"]
text_link = bill["bill"]["texts"][-1]["url"]
sponsors = []
for person in bill["bill"]["sponsors"]:
    sponsors.append(person["people_id"])
roll_call_ids = []
for vote in bill["bill"]["votes"]:
    roll_call_ids.append(vote["roll_call_id"])
topics = []
for topic in bill["bill"]["subjects"]:
    topics.append(topic["subject_name"])
example_bill = {
    "ID": bill_id,
    "Name": bill_name,
    "Number": bill_number,
    "State": bill_state,
    "Status": status,
    "Description": bill_desc,
    "Text": text_link,
    "Topics": f"{topics}",
    "Sponsors": f"{sponsors}",
    "Votes": f"{roll_call_ids}",
}
example_bill

{'ID': 1830799,
 'Name': 'Privacy of Biometric Identifiers & Data',
 'Number': 'HB1130',
 'State': 'CO',
 'Status': 'Passed',
 'Description': "Concerning protecting the privacy of an individual's biometric data.",
 'Text': 'https://legiscan.com/CO/text/HB1130/id/3002852',
 'Topics': "['Civil Law', 'Public Health']",
 'Sponsors': '[22223, 22227, 19205, 16509, 22219, 22220, 20127, 22921, 24873, 25044, 20148, 23958, 24384, 18721, 22225, 23956, 20159, 23967, 23046, 23954, 23949, 23959, 25123, 20025, 24407, 22231, 24966, 20024, 20139, 20032, 20142, 20143, 23963, 18723, 21455, 18712, 20126, 19048, 18719, 20131, 18726, 14268, 20029, 23104, 18713, 23972, 3323, 16578, 21293]',
 'Votes': '[1384592, 1388405, 1388406, 1388407, 1414991, 1414992, 1414993, 1414994, 1427070, 1427883, 1431305, 1431622, 1431623]'}

Much better, let's walk through the input folders and do that to every single one of our bills, then add them to a DataFrame so we can save it out to a database later.


In [ ]:
with lzma.open(r"H:\Programming\Big Input Data Stuff\LegiScan\cleaned_input\bills.pkl.xz", 'r') as f:
    bills = pd.read_pickle(f)
statesDict = {}
statuses = [
    "N\\A",
    "Intro",
    "Engrossed",
    "Enrolled",
    "Passed",
    "Vetoed",
    "Failed",
    "Override",
    "Chaptered",
    "Refer",
    "Report Pass",
    "Report DNP",
    "Draft",
]
with open("./processed_bills.pkl", 'rb') as f:
    processedBills = pickle.load(f)

new_bills = []

def process_bill_file(file):
    processed_bill = ({})
    with open(file, 'r') as f:
        bill = json.loads(f.read())
        statesDict[bill["bill"]["state_id"]] = bill["bill"]["state"]
        bill_id = bill["bill"]["bill_id"]
        status = statuses[bill["bill"]["status"]]
        bill_name = bill["bill"]["title"]
        bill_number = bill["bill"]["bill_number"]
        bill_state = bill["bill"]["state"]
        bill_desc = bill["bill"]["description"]
        if len(bill["bill"]["texts"]) != 0:
            text_link = bill["bill"]["texts"][-1]["url"]
        else:
            text_link = ""
        sponsors = []
        for person in bill["bill"]["sponsors"]:
            sponsors.append(person["people_id"])
        roll_call_ids = []
        for vote in bill["bill"]["votes"]:
            roll_call_ids.append(vote["roll_call_id"])
        topics = []
        for topic in bill["bill"]["subjects"]:
            topics.append(topic["subject_name"])
        processed_bill = ({
                "ID": bill_id,
                "Name": bill_name,
                "Number": bill_number,
                "State": bill_state,
                "Description": bill_desc,
                "Status": status,
                "Text": text_link,
                "Topics": f"{topics}",
                "Sponsors": f"{sponsors}",
                "Votes": f"{roll_call_ids}",
            })
    return processed_bill

for f in os.walk(r"H:\Programming\Big Input Data Stuff\LegiScan\input"):
    if "bill" in f[0]:
        with open(f[0][:-4] + "hash.md5") as hash_f:
            folder_hash = hash_f.read()
        if folder_hash not in processedBills:
            count = len([name for name in os.listdir(f[0]) if os.path.isfile(os.path.join(f[0], name))])
            chunk_size = math.ceil(count/processors)
            with ThreadPool(processes=processors) as pool:
                actual_files = []
                for file in f[-1]:
                    if ".json" in file:
                        file_name = os.path.join(f[0], file)
                        actual_files.append(file_name)
                session_bills = pool.map(process_bill_file, actual_files, chunksize=chunk_size)
            new_bills.extend(session_bills)
            processedBills.add(folder_hash)

if new_bills:
    new_bills_df = pd.DataFrame.from_records(new_bills)
    new_bills_df.drop_duplicates(inplace=True)
    new_bills_df.set_index("ID", inplace=True)
    bills.update(new_bills_df)
    bills = pd.concat([bills, new_bills_df]).drop_duplicates()

with open("./processed_bills.pkl", 'wb') as f:
    pickle.dump(processedBills, f)

In [5]:
bills

,Name,Number,State,Description,Status,Text,Topics,Sponsors,Votes
ID,,,,,,,,,
132957,Amend Amt Of Base Student Allocation,HB1,AK,An Act relating to the base student allocation...,Intro,https://legiscan.com/AK/text/HB1/id/365510,"['Education', 'Public Finance', 'School Distri...","[6033, 6034]",[]
132966,Municipal Tax Exemptions,HB10,AK,An Act relating to mandatory exemptions from m...,Passed,https://legiscan.com/AK/text/HB10/id/367679,"['Age Of Persons', 'Disabilities', 'Law Enforc...","[6033, 6054, 6053, 6052, 6051, 6050, 6046, 604...",[]
133056,"Joseph C Williams, Sr., Coastal Trail",HB100,AK,"An Act relating to the Joseph C. Williams, Sr....",Passed,https://legiscan.com/AK/text/HB100/id/392326,"['Highways', 'Place Names', 'Trails', 'Transpo...",[6067],[]
133057,Exemptions: Life Insurance; Annuities,HB101,AK,An Act increasing the value of life insurance ...,Passed,https://legiscan.com/AK/text/HB101/id/392605,"['Civil Procedure', 'Death', 'Debt', 'Insuranc...","[6034, 6061]",[]
133058,Uniform Commercial Code/liens/contracts,HB102,AK,An Act relating to the Uniform Commercial Code...,Passed,https://legiscan.com/AK/text/HB102/id/392839,"['Animals', 'Business', 'Court Rules', 'Credit...",[],[]
...,...,...,...,...,...,...,...,...,...
1936581,NCSL 50th Anniversary.,SJ0007,WY,A JOINT RESOLUTION to recognize and commend th...,Failed,https://legiscan.com/WY/text/SJ0007/id/3074019,[],"[19384, 24161, 14230, 8713, 20193, 14414, 1422...","[1477425, 1479160, 1499636]"
1939111,Political expenditures.,SJ0008,WY,A JOINT RESOLUTION requesting Congress to prop...,Failed,https://legiscan.com/WY/text/SJ0008/id/3076742,[],"[19384, 24160, 25310, 23879, 25311, 25312, 253...","[1482970, 1485987, 1501622]"
1942332,Individual right to privacy-constitutional ame...,SJ0009,WY,A JOINT RESOLUTION proposing to amend the Wyom...,Failed,https://legiscan.com/WY/text/SJ0009/id/3083423,[],"[19384, 22676, 20193, 8675, 10602]",[]


Alright, now for the next part. We need to take votes for each bill and pull detailed information from the voting data provided by LegiScan. We'll handle this similarly to the way we handled the bills themselves.

First let's take a look at one of these vote files to see what we will and won't need.


In [6]:
with open(r"H:\Programming\Big Input Data Stuff\LegiScan\input\CO\2024-2024_Regular_Session\vote\1414993.json", 'r') as f:
    print(json.dumps(json.loads(f.read()), indent=2))

{
  "roll_call": {
    "roll_call_id": 1414993,
    "bill_id": 1830799,
    "date": "2024-02-14",
    "desc": "House Judiciary: Adopt amendment L.004 (Attachment E)",
    "yea": 11,
    "nay": 0,
    "nv": 0,
    "absent": 0,
    "total": 11,
    "passed": 1,
    "chamber": "H",
    "chamber_id": 21,
    "votes": [
      {
        "people_id": 18721,
        "vote_id": 1,
        "vote_text": "Yea"
      },
      {
        "people_id": 18723,
        "vote_id": 1,
        "vote_text": "Yea"
      },
      {
        "people_id": 20138,
        "vote_id": 1,
        "vote_text": "Yea"
      },
      {
        "people_id": 20139,
        "vote_id": 1,
        "vote_text": "Yea"
      },
      {
        "people_id": 21549,
        "vote_id": 1,
        "vote_text": "Yea"
      },
      {
        "people_id": 22219,
        "vote_id": 1,
        "vote_text": "Yea"
      },
      {
        "people_id": 22220,
        "vote_id": 1,
        "vote_text": "Yea"
      },
      {
        "people_i

Again, we're seeing a lot of information we don't necessarily care about here. Let's isolate the information we do care about.

In [7]:
with open(r"H:\Programming\Big Input Data Stuff\LegiScan\input\CO\2024-2024_Regular_Session\vote\1414993.json", 'r') as f:
    vote = json.loads(f.read())

vote_id = vote["roll_call"]["roll_call_id"]
bill_vote_id = vote["roll_call"]["bill_id"]
vote_desc = vote["roll_call"]["desc"]
vote_passed = bool(vote["roll_call"]["passed"])
actual_votes = []
for v in vote["roll_call"]["votes"]:
    actual_votes.append((v["people_id"], v["vote_text"]))
example_vote = {
    "ID": vote_id,
    "Bill ID": bill_vote_id,
    "Description": vote_desc,
    "Passed": vote_passed,
    "Votes": f"{actual_votes}",
}
example_vote

{'ID': 1414993,
 'Bill ID': 1830799,
 'Description': 'House Judiciary: Adopt amendment L.004 (Attachment E)',
 'Passed': True,
 'Votes': "[(18721, 'Yea'), (18723, 'Yea'), (20138, 'Yea'), (20139, 'Yea'), (21549, 'Yea'), (22219, 'Yea'), (22220, 'Yea'), (23953, 'Yea'), (23954, 'Yea'), (23965, 'Yea'), (24385, 'Yea')]"}

Nice, let's walk through the input folders and do that to every single one of our votes, just like we did with the bills.


In [ ]:
with lzma.open(r"H:\Programming\Big Input Data Stuff\LegiScan\cleaned_input\votes.pkl.xz", 'r') as f:
    votes = pd.read_pickle(f)
with open("./processed_votes.pkl", 'rb') as f:
    processedVotes = pickle.load(f)

new_votes = []

def process_vote_file(file):
    processed_vote = ({})
    with open(file, 'r') as f:
        try:
            vote = json.loads(f.read())
        except JSONDecodeError:
            print(f"File {f} failed to decode as JSON")
        vote_id = vote["roll_call"]["roll_call_id"]
        bill_vote_id = vote["roll_call"]["bill_id"]
        vote_desc = vote["roll_call"]["desc"]
        vote_passed = bool(vote["roll_call"]["passed"])
        actual_votes = []
        for v in vote["roll_call"]["votes"]:
            actual_votes.append((v["people_id"], v["vote_text"]))
        processed_vote = ({
            "ID": vote_id,
            "Bill ID": bill_vote_id,
            "Description": vote_desc,
            "Passed": vote_passed,
            "Votes": f"{actual_votes}",
        })
    return processed_vote
    
for f in os.walk(r"H:\Programming\Big Input Data Stuff\LegiScan\input"):
    if "vote" in f[0]:
        with open(f[0][:-4] + "hash.md5") as hash_f:
            folder_hash = hash_f.read()
        if folder_hash not in processedVotes:
            count = len([name for name in os.listdir(f[0]) if os.path.isfile(os.path.join(f[0], name))])
            chunk_size = math.ceil(count/processors)
            with ThreadPool(processes=processors) as pool:
                actual_files = []
                for file in f[-1]:
                    if ".json" in file:
                        file_name = os.path.join(f[0], file)
                        actual_files.append(file_name)
                session_votes = pool.map(process_vote_file, actual_files, chunksize=chunk_size)
            new_votes.extend(session_votes)
            processedVotes.add(folder_hash)

if new_votes:
    new_votes_df = pd.DataFrame.from_records(new_votes)
    new_votes_df.drop_duplicates(inplace=True)
    new_votes_df.set_index("ID", inplace=True)
    votes.update(new_votes_df)
    votes = pd.concat([votes, new_votes_df]).drop_duplicates()

with open("./processed_votes.pkl", 'wb') as f:
    pickle.dump(processedVotes, f)

In [9]:
votes

,Bill ID,Description,Passed,Votes
ID,,,,
306479,454312,Senate: <pre> SR 1 Final Passage,True,"[(6044, 'Yea'), (6061, 'Yea'), (6064, 'Yea'), ..."
306480,472178,Senate: CSHB 84(FIN)(efd am S) Third Reading -...,True,"[(6044, 'Yea'), (6061, 'Yea'), (6064, 'Yea'), ..."
306482,472178,House: Concur,True,"[(6034, 'Yea'), (6035, 'Yea'), (6037, 'Yea'), ..."
306483,545632,House: Special Order of Business,True,"[(6034, 'Yea'), (6035, 'Yea'), (6037, 'Yea'), ..."
306484,445345,House: Third Reading Final Passage,True,"[(6034, 'Yea'), (6035, 'Yea'), (6037, 'Yea'), ..."
...,...,...,...,...
1507345,1926046,SF0127VT001 Amendment Reconsideration Motion P...,True,"[(8713, 'Nay'), (8725, 'Nay'), (10612, 'Yea'),..."
1507351,1939132,SF0169JC002 Adopted SF0169JC002: 19-9-3-0-0,True,"[(8641, 'Yea'), (8663, 'Yea'), (8675, 'Nay'), ..."
1507352,1939132,SF0169JC002 Adopted SF0169JC002: 38-15-9-0-0,True,"[(8713, 'Yea'), (8725, 'Nay'), (10612, 'Nay'),..."


Amazing, now let's round it out by doing the same thing to the people data, starting out with a quick check up on what data exists.


In [10]:
with open(r"H:\Programming\Big Input Data Stuff\LegiScan\input\NY\2023-2024_General_Assembly\people\22367.json", 'r') as f:
    print(json.dumps(json.loads(f.read()), indent=2))

{
  "person": {
    "people_id": 22367,
    "person_hash": "fz1bgi6f",
    "party_id": "1",
    "state_id": 32,
    "party": "D",
    "role_id": 1,
    "role": "Rep",
    "name": "Zohran Kwame Mamdani",
    "first_name": "Zohran",
    "middle_name": "Kwame",
    "last_name": "Mamdani",
    "suffix": "",
    "nickname": "",
    "district": "HD-036",
    "ftm_eid": 50109025,
    "votesmart_id": 192156,
    "opensecrets_id": "",
    "knowwho_pid": 740165,
    "ballotpedia": "Zohran_Kwame_Mamdani",
    "bioguide_id": "",
    "committee_sponsor": 0,
    "committee_id": 0,
    "state_federal": 0
  }
}


Once again we'll pare it down to only what's relevant to this project.

In [11]:
with open(r"H:\Programming\Big Input Data Stuff\LegiScan\input\NY\2023-2024_General_Assembly\people\22367.json", 'r') as f:
    person = json.loads(f.read())
person_id = person["person"]["people_id"]
party = person["person"]["party"]
role = person["person"]["role"]
name = person["person"]["name"]
state = statesDict[person["person"]["state_id"]]
district = person["person"]["district"]
example_person = {
    "ID": person_id,
    "Name": name,
    "Party": party,
    "Role": role,
    "State": state,
    "District": district,
}
example_person

{'ID': 22367,
 'Name': 'Zohran Kwame Mamdani',
 'Party': 'D',
 'Role': 'Rep',
 'State': 'NY',
 'District': 'HD-036'}

And lastly, let's run through the whole dataset, just like we did with everything else.

In [ ]:
people = []

def process_people_file(file):
    processed_person = ({})
    with open(file, 'r') as f:
        person = json.loads(f.read())
        person_id = person["person"]["people_id"]
        party = person["person"]["party"]
        role = person["person"]["role"]
        name = person["person"]["name"]
        state = statesDict[person["person"]["state_id"]]
        district = person["person"]["district"]
        processed_person = ({
            "ID": person_id,
            "Name": name,
            "Party": party,
            "Role": role,
            "State": state,
            "District": district,
        })
    return processed_person

for f in os.walk(r"H:\Programming\Big Input Data Stuff\LegiScan\input"):
    if "people" in f[0]:
        count = len([name for name in os.listdir(f[0]) if os.path.isfile(os.path.join(f[0], name))])
        chunk_size = math.ceil(count/processors)
        with ThreadPool(processes=processors) as pool:
            actual_files = []
            for file in f[-1]:
                if ".json" in file:
                    file_name = os.path.join(f[0], file)
                    actual_files.append(file_name)
            session_people = pool.map(process_people_file, actual_files, chunksize=chunk_size)
        people.extend(session_people)

people = pd.DataFrame.from_records(people)
people.set_index("ID", inplace=True)


In [13]:
people

,Name,Party,Role,State,District
ID,,,,,
6033,Carl Gatto,R,Rep,AK,HD-013
6034,Robert Lynn,R,Rep,AK,HD-026
6035,Max Gruenberg,D,Rep,AK,HD-016
6036,Nancy Dahlstrom,R,Rep,AK,HD-018
6037,Wes Keller,R,Rep,AK,HD-010
...,...,...,...,...,...
8702,Select Water,,Jnt,WY,
8706,Select School Facilities,,Jnt,WY,
8713,Bob Nicholas,R,Rep,WY,HD-007


Some states appear to have councils as people? Not sure why, but let's just remove all of those real quick.


In [14]:
people = people[people["District"] != ""]
people

,Name,Party,Role,State,District
ID,,,,,
6033,Carl Gatto,R,Rep,AK,HD-013
6034,Robert Lynn,R,Rep,AK,HD-026
6035,Max Gruenberg,D,Rep,AK,HD-016
6036,Nancy Dahlstrom,R,Rep,AK,HD-018
6037,Wes Keller,R,Rep,AK,HD-010
...,...,...,...,...,...
8663,Charles Scott,R,Sen,WY,SD-030
8675,Cale Case,R,Sen,WY,SD-025
8679,Dan Dockstader,R,Sen,WY,SD-016


Brilliant, now let's close it out by saving these to some pickles (and compress them) so we can analyze it later without having to do all the cleaning again.


In [15]:
bills.to_pickle(r"H:\Programming\Big Input Data Stuff\LegiScan\cleaned_input\bills.pkl.xz")
people.to_pickle(r"H:\Programming\Big Input Data Stuff\LegiScan\cleaned_input\people.pkl.xz")
votes.to_pickle(r"H:\Programming\Big Input Data Stuff\LegiScan\cleaned_input\votes.pkl.xz")

Last and perhaps least, let's save the runtime of this whole notebook to our timings file so we can know if there is new data next time this notebook is run.


In [16]:
with open("./times.txt", "r+") as f:
    old_data = f.readlines()
    if len(old_data) >= 2:
        old_data[
            1
        ] = f"Last clean happened at #{datetime.now(timezone.utc).strftime(f'%d/%m/%y %H:%M:%S')}\n"
    else:
        old_data.append(
            f"Last clean happened at #{datetime.now(timezone.utc).strftime(f'%d/%m/%y %H:%M:%S')}\n"
        )
    f.seek(0)
    for line in old_data:
        f.write(line)
    f.truncate()